In [1]:
import pandas as pd
import spacy
from spacy.pipeline import EntityRuler
import tqdm
import glob
import os

In [2]:
import spacy
print(f"spaCy version: {spacy.__version__}")
print(f"CUDA available: {spacy.prefer_gpu()}")
# print(f"GPU device count: {spacy.util.get_gpu_count()}")

# Load spaCy model
spacy.prefer_gpu()
nlp = spacy.load("en_core_web_sm") 

# Add your custom EntityRuler
ruler = nlp.add_pipe("entity_ruler", before="ner")

spaCy version: 3.8.5
CUDA available: True


In [3]:
#Init Variables for csv names
year="2024"
# month="01"
cwd=os.getcwd()
month=os.path.basename(cwd)
# print(f"{month}")

#special_identifier='_xfin_amt_sep_spi_ama'
special_identifier='_PLOTS_1' #for csv output
# Construct the directory name
output_directory = f"batch{special_identifier}"

services = [
    "Comcast", "Airline", "Healthcare", "Trains", "Banks", "United States",
    "ER", "Youtube", "Reddit", "Netflix",
    "Xfinity", "Amtrak", "Septa", "Spirit", "American",
    "Disney"
]

complaint_patterns = [{"label": "SERVICE", "pattern": service} for service in services]
ruler.add_patterns(complaint_patterns)

In [4]:
# Load data
df = pd.read_csv(f"../../FILTERED_SUBMISSIONS_bad/RS_{year}-{month}.csv", names=["id", "date", "title", "author", "url", "content", "post_id", "timestamp", "subreddit"])

# df.head()

In [5]:
# Build regex pattern for fast searching (case insensitive)
pattern = r'|'.join(services)

# Filter rows where content mentions any service
filtered_df = df[df['content' ].str.contains(pattern, case=False, na=False) |
                df['subreddit'].str.contains(pattern, case=False, na=False) ]

print(f"Original size: {len(df)}")
print(f"Filtered size: {len(filtered_df)}")

Original size: 486721
Filtered size: 436221


In [6]:
# neg_words = ["bad", "terrible", "lazy", "worst", "awful", "scam", "horrible", "broken", "slow"]
neg_words = [
            "bad", "terrible", "lazy", "worst", "awful", 
            "scam", "horrible", "broken", "slow", "usless", 
            "sucks", "ripoff", "expensive", "painful", "crumy",
            "pointless", "greedy", "fake", "disappointing", "mess",
            "nightmare"
            ]

pattern_neg = r'|'.join(neg_words)

complaint_df = filtered_df[
    filtered_df['content'].str.contains(pattern_neg, case=False, na=False)
]

# complaint_df.head()

In [7]:
# complaint_df.info()

In [ ]:
from tqdm.notebook import tqdm

batch_size = 5000
max_length = 1000  # Skip posts longer than this
skipped_posts = []

# Check if the directory exists, and create it if it doesn't
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    print(f"Created directory: {output_directory}")
else:
    print(f"Directory already exists: {output_directory}")


for start in tqdm(range(0, len(df), batch_size)):
    end = start + batch_size
    batch = df.iloc[start:end].copy()

    # Only keep rows with short enough content
    batch = batch[batch['content'].str.len() < max_length]

    texts = batch['content'].tolist()
    ner_results = []

    for doc, row in zip(nlp.pipe(texts, batch_size=50), batch.itertuples()):
        try:
            ents = [(ent.text, ent.label_) for ent in doc.ents]
        except Exception as e:
            print(f"Skipping post {row.post_id} due to error: {e}")
            skipped_posts.append(row.post_id)
            ents = []

        ner_results.append(ents)

    batch['entities'] = ner_results

    # Construct the full file path
    file_name = f'ner_results_batch_{year}.{month}_{start}{special_identifier}.csv'
    full_file_path = os.path.join(output_directory, file_name)

    # Save the DataFrame to the CSV file in the new directory
    batch.to_csv(full_file_path, index=False)

# Save skipped post IDs
pd.Series(skipped_posts).to_csv('skipped_posts.csv', index=False)

print("NER complete.")


Created directory: batch_PLOTS_1


  0%|          | 0/98 [00:00<?, ?it/s]

In [ ]:
import glob

files = glob.glob(os.path.join(output_directory, f"ner_results_batch_*{special_identifier}.csv"))
dfs = [pd.read_csv(os.path.join(f)) for f in files]

final_ner_df = pd.concat(dfs, ignore_index=True)
final_ner_df.to_csv(os.path.join(output_directory, f'ner_results_append_sum_{year}.{month}{special_identifier}.csv', index=False)

In [ ]:
from collections import Counter
import ast
from tqdm.notebook import tqdm

entity_counter = Counter()

for entities in tqdm(final_ner_df['entities'], desc="Processing Entities"):
    try:
        ents = ast.literal_eval(entities)
        entity_counter.update([e[0] for e in ents])
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing entities: {e} for input: {entities}")
        continue  # Skip to the next item if there's an error

print(entity_counter.most_common(100))

In [ ]:
subreddit_counter = final_ner_df['subreddit'].value_counts()

print(subreddit_counter.head(20))

In [ ]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

analyzer = SentimentIntensityAnalyzer()

In [ ]:
from tqdm.notebook import tqdm
tqdm.pandas()

def get_sentiment(text):
    if isinstance(text, str):
        return analyzer.polarity_scores(text)['compound']
    return 0.0  # Neutral if empty or NaN

# final_ner_df['sentiment'] = final_ner_df['content'].apply(get_sentiment)
final_ner_df['sentiment'] = final_ner_df['content'].progress_apply(get_sentiment)

In [ ]:
final_ner_df.to_csv(os.path.join(output_directory, f'ner_results_append_sum_{year}_{month}_sentiment{special_identifier}.csv', index=False)

In [ ]:
# Construct the plot output directory
plot_output_directory = f"batch{special_identifier}/PLOTS"

# Create the PLOTS directory if it doesn't exist
os.makedirs(plot_output_directory, exist_ok=True)

In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt

# Extract SERVICE entities and their sentiment and timestamp
service_post_counts = []

for row in final_ner_df.itertuples():
    try:
        ents = ast.literal_eval(row.entities)
        # Filter for 'SERVICE' entities that are in your predefined list
        services_in_post = [e[0] for e in ents if e[1] == 'SERVICE' and e[0] in services]
        for service in services_in_post:
            service_post_counts.append((service, row.timestamp)) # Include timestamp
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing entities: {e} for row with index {row.Index}")
        continue

# Create a DataFrame to store service and timestamp
service_timestamp_df = pd.DataFrame(service_post_counts, columns=['service', 'timestamp'])

# Convert timestamp to a datetime object if it's not already
if not pd.api.types.is_datetime64_any_dtype(service_timestamp_df['timestamp']):
    service_timestamp_df['timestamp'] = pd.to_datetime(service_timestamp_df['timestamp'], unit='s') # Assuming timestamp is in seconds since epoch, adjust 'unit' if needed

# Extract the date part from the timestamp
service_timestamp_df['date'] = service_timestamp_df['timestamp'].dt.date

# Group by service and date and count the number of posts
post_counts_per_day = service_timestamp_df.groupby(['service', 'date']).size().reset_index(name='post_count')

# --- Plotting ---
plt.figure(figsize=(12, 6))  # Adjust figure size as needed

for service in services:
    service_data = post_counts_per_day[post_counts_per_day['service'] == service]
    plt.plot(service_data['date'], service_data['post_count'], label=service)

plt.xlabel("Date")
plt.ylabel("Post Count")
plt.title("Frequency of Posts per Service per Day")
plt.legend(title="Entity")
plt.grid(True)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
# plt.show()

# Save the plot to the specified directory
plot_filename = os.path.join(plot_output_directory, "post_frequency_per_service.png")
plt.savefig(plot_filename)
plt.close() # Close the plot to prevent display in notebook if running in batch

print(f"Plot saved to: {plot_filename}")

In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
from itertools import cycle

# Extract SERVICE entities and their sentiment and timestamp
service_post_counts = []

for row in final_ner_df.itertuples():
    try:
        ents = ast.literal_eval(row.entities)
        # Filter for 'SERVICE' entities that are in your predefined list
        services_in_post = [e[0] for e in ents if e[1] == 'SERVICE' and e[0] in services]
        for service in services_in_post:
            service_post_counts.append((service, row.timestamp)) # Include timestamp
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing entities: {e} for row with index {row.Index}")
        continue

# Create a DataFrame to store service and timestamp
service_timestamp_df = pd.DataFrame(service_post_counts, columns=['service', 'timestamp'])

# Convert timestamp to a datetime object if it's not already
if not pd.api.types.is_datetime64_any_dtype(service_timestamp_df['timestamp']):
    service_timestamp_df['timestamp'] = pd.to_datetime(service_timestamp_df['timestamp'], unit='s') # Assuming timestamp is in seconds since epoch, adjust 'unit' if needed

# Extract the date part from the timestamp
service_timestamp_df['date'] = service_timestamp_df['timestamp'].dt.date

# Group by service and date and count the number of posts
post_counts_per_day = service_timestamp_df.groupby(['service', 'date']).size().reset_index(name='post_count')

# --- Overlapping Bar Chart Plotting ---
plt.figure(figsize=(14, 7))  # Adjust figure size as needed

colors = cycle(plt.cm.tab10.colors) # Cycle through a set of distinct colors
opacity = 0.6

for service in services:
    service_data = post_counts_per_day[post_counts_per_day['service'] == service]
    color = next(colors)
    plt.bar(service_data['date'], service_data['post_count'], label=service, color=color, alpha=opacity)

plt.xlabel("Date")
plt.ylabel("Post Count")
plt.title("Frequency of Posts per Service per Day (Overlapping Bar Charts)")
plt.legend(title="Entity")
plt.grid(axis='y', linestyle='--')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
# plt.show()

# Save the plot to the specified directory
plot_filename = os.path.join(plot_output_directory, "post_frequency_per_service_barchart.png")
plt.savefig(plot_filename)
plt.close() # Close the plot to prevent display in notebook if running in batch

print(f"Plot saved to: {plot_filename}")

In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt

# Extract SERVICE entities and their sentiment and timestamp
service_post_counts_all = []

for row in final_ner_df.itertuples():
    try:
        ents = ast.literal_eval(row.entities)
        services_in_post = [e[0] for e in ents if e[1] == 'SERVICE' and e[0] in services]
        for service in services_in_post:
            service_post_counts_all.append((service, row.timestamp))
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing entities: {e} for row with index {row.Index}")
        continue

service_timestamp_df_all = pd.DataFrame(service_post_counts_all, columns=['service', 'timestamp'])

if not pd.api.types.is_datetime64_any_dtype(service_timestamp_df_all['timestamp']):
    service_timestamp_df_all['timestamp'] = pd.to_datetime(service_timestamp_df_all['timestamp'], unit='s')

service_timestamp_df_all['date'] = service_timestamp_df_all['timestamp'].dt.date

post_counts_per_day_all = service_timestamp_df_all.groupby(['service', 'date']).size().reset_index(name='post_count')

# Calculate total post counts per service
total_posts_per_service = post_counts_per_day_all.groupby('service')['post_count'].sum().sort_values(ascending=False)

# Divide services into three groups
n_services = len(services)
third = n_services // 3
top_third_services = total_posts_per_service.index[:third]
middle_third_services = total_posts_per_service.index[third:2*third]
bottom_third_services = total_posts_per_service.index[2*third:]

# --- Plotting ---
fig, axes = plt.subplots(3, 1, figsize=(14, 12), sharex=True)
fig.suptitle("Frequency of Posts per Service per Day", fontsize=16, y=1.02)

def plot_service_group(ax, services_to_plot, title):
    for service in services_to_plot:
        service_data = post_counts_per_day_all[post_counts_per_day_all['service'] == service]
        ax.bar(service_data['date'], service_data['post_count'], label=service, alpha=0.7)
    ax.set_ylabel("Post Count")
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=45, labelbottom=True)
    ax.legend(title="Entity")
    ax.grid(axis='y', linestyle='--')

plot_service_group(axes[0], top_third_services, f"Top {len(top_third_services)} Most Frequent Services")
plot_service_group(axes[1], middle_third_services, f"Next {len(middle_third_services)} Most Frequent Services")
plot_service_group(axes[2], bottom_third_services, f"Remaining {len(bottom_third_services)} Services")

fig.text(0.5, 0.01, 'Date', ha='center')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.show()

# Save the plot to the specified directory
plot_filename = os.path.join(plot_output_directory, "post_frequency_per_service_volatility.png")
plt.savefig(plot_filename)
plt.close() # Close the plot to prevent display in notebook if running in batch

print(f"Plot saved to: {plot_filename}")

In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
from itertools import cycle

# Extract SERVICE entities and their sentiment and timestamp
entity_sentiments_over_time = []

for row in final_ner_df.itertuples():
    try:
        ents = ast.literal_eval(row.entities)
        date = pd.to_datetime(row.timestamp, unit='s').date() # Extract date
        for ent_text, ent_label in ents:
            if ent_label == 'SERVICE' and ent_text in services:
                entity_sentiments_over_time.append((ent_text, date, row.sentiment))
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing entities: {e} for row with index {row.Index}")
        continue

# Create a DataFrame for entity sentiment over time
entity_sentiment_df = pd.DataFrame(entity_sentiments_over_time, columns=['service', 'date', 'sentiment'])

# Calculate average sentiment per entity per day
avg_sentiment_per_day = entity_sentiment_df.groupby(['service', 'date'])['sentiment'].mean().reset_index()

# --- Plotting Sentiment vs. Date ---
plt.figure(figsize=(14, 7))
colors = cycle(plt.cm.tab10.colors)

for service in services:
    service_data = avg_sentiment_per_day[avg_sentiment_per_day['service'] == service]
    color = next(colors)
    plt.plot(service_data['date'], service_data['sentiment'], label=service, color=color)

plt.xlabel("Date")
plt.ylabel("Average Sentiment")
plt.title("Sentiment of Posts Over Time by Entity")
plt.legend(title="Entity") # Changed legend title here
plt.grid(True)
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
# plt.show()

# Save the plot to the specified directory
plot_filename = os.path.join(plot_output_directory, "post_sentiment_by_entity.png")
plt.savefig(plot_filename)
plt.close() # Close the plot to prevent display in notebook if running in batch

print(f"Plot saved to: {plot_filename}")

In [ ]:
import pandas as pd
import ast
import matplotlib.pyplot as plt
from itertools import cycle
from tqdm.notebook import tqdm

# Extract SERVICE entities and their sentiment and timestamp
entity_sentiments_over_time = []

for row in tqdm(final_ner_df.itertuples(), total=len(final_ner_df), desc="Extracting Entity Sentiments"):
    try:
        ents = ast.literal_eval(row.entities)
        date = pd.to_datetime(row.timestamp, unit='s').date() # Extract date
        for ent_text, ent_label in ents:
            if ent_label == 'SERVICE' and ent_text in services:
                entity_sentiments_over_time.append((ent_text, date, row.sentiment))
    except (SyntaxError, ValueError) as e:
        print(f"Error parsing entities: {e} for row with index {row.Index}")
        continue

# Create a DataFrame for entity sentiment over time
entity_sentiment_df = pd.DataFrame(entity_sentiments_over_time, columns=['service', 'date', 'sentiment'])

# Calculate average sentiment per entity per day
avg_sentiment_per_day = entity_sentiment_df.groupby(['service', 'date'])['sentiment'].mean().reset_index()

# Calculate volatility (standard deviation of sentiment) per service
sentiment_volatility = avg_sentiment_per_day.groupby('service')['sentiment'].std().sort_values(ascending=False)

# Divide services into three groups based on volatility
n_services = len(services)
third = n_services // 3
high_volatility_services = sentiment_volatility.index[:third].tolist()
medium_volatility_services = sentiment_volatility.index[third:2*third].tolist()
low_volatility_services = sentiment_volatility.index[2*third:].tolist()

# --- Plotting Sentiment vs. Date by Volatility ---
fig, axes = plt.subplots(3, 1, figsize=(16, 12), sharex=True)
fig.suptitle("Average Sentiment of Posts Over Time by Entity Volatility", fontsize=16, y=1.02)

def plot_sentiment_group(ax, services_to_plot, title):
    colors = cycle(plt.cm.tab10.colors)
    for service in services_to_plot:
        service_data = avg_sentiment_per_day[avg_sentiment_per_day['service'] == service]
        color = next(colors)
        ax.plot(service_data['date'], service_data['sentiment'], label=service, color=color)
    ax.set_ylabel("Average Sentiment")
    ax.set_title(title)
    ax.tick_params(axis='x', rotation=45, labelbottom=True)
    ax.legend(title="Entity", bbox_to_anchor=(1.05, 1), loc='upper left')
    ax.grid(True)

plot_sentiment_group(axes[0], high_volatility_services, f"High Volatility Entities")
plot_sentiment_group(axes[1], medium_volatility_services, f"Medium Volatility Entities")
plot_sentiment_group(axes[2], low_volatility_services, f"Low Volatility Entities")

fig.text(0.5, 0.01, 'Date', ha='center')
plt.tight_layout(rect=[0, 0.03, 1, 0.95])
# plt.show()

# Save the plot to the specified directory
plot_filename = os.path.join(plot_output_directory, "post_sentiment_by_entity_by_volatility.png")
plt.savefig(plot_filename)
plt.close() # Close the plot to prevent display in notebook if running in batch

print(f"Plot saved to: {plot_filename}")